In [21]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# magic command for matplotlib to enable printing directly in the jupyter notebook
%matplotlib inline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import plot_confusion_matrix
#from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.utils import resample,shuffle
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn import metrics



In [2]:
# PATHS

# get working directory
path_working = Path.cwd()

path_data = str(path_working) + "\\" + "data" + "\\"

#print(path_data)


In [3]:
def create_dataframe (path):
    # Get the files from the path provided in the OP
    files = Path(path).glob('*.csv')  # .rglob to create iterator for subdirectories
    
    #Find all csv files in directory and concatenate them
    dfs = list()
    names = list()
    for file in files:
        data = pd.read_csv(file)
        # .stem is method for pathlib objects to get the filename without the extension
        # create another column in dataframe for filename
        data['file'] = file.stem
        names.append(file.stem)
        dfs.append(data)
        df = pd.concat(dfs, ignore_index=True)
    
    return df


In [4]:
df_raw = create_dataframe(path_data)

In [5]:
df_raw

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       relay1_log  relay2_log  relay3_log  relay4_log  snort_log1  snort_log2  \
0               0           0           0           0           0           0   
1               0           0           0           0           0           0   
2               0           0           0           0           0           0   
3               0           0           0           0           0           0   
4               0           0           0           0           0           0   
...           ...         ...         ...         ...         ...         ...   
78372           0           0           0           0           0           0   
78373           0           0           0           0           0           0   
78374           0           0           0           0           0           0   
78375           0           0           0           0           0           0   
78376           0           0           0           0           0           0   

       snort_log3  snort_log4    marker   file  
0               0           0  NoEvents  data1  
1               0           0  NoEvents  data1  
2               0           0  NoEvents  data1  
3               0           0  NoEvents  data1  
4               0           0  NoEvents  data1  
...           ...         ...       ...    ...  
78372           0           0   Natural  data9  
78373           0           0   Natural  data9  
78374           0           0   Natural  data9  
78375           0           0   Natural  data9  
78376           0           0   Natural  data9  

[78377 rows x 130 columns]

In [6]:
#drop last column taht shows csv filename source:
del df_raw[df_raw.columns[-1]]

df_raw

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       control_panel_log4  relay1_log  relay2_log  relay3_log  relay4_log  \
0                       0           0           0           0           0   
1                       0           0           0           0           0   
2                       0           0           0           0           0   
3                       0           0           0           0           0   
4                       0           0           0           0           0   
...                   ...         ...         ...         ...         ...   
78372                   0           0           0           0           0   
78373                   0           0           0           0           0   
78374                   0           0           0           0           0   
78375                   0           0           0           0           0   
78376                   0           0           0           0           0   

       snort_log1  snort_log2  snort_log3  snort_log4    marker  
0               0           0           0           0  NoEvents  
1               0           0           0           0  NoEvents  
2               0           0           0           0  NoEvents  
3               0           0           0           0  NoEvents  
4               0           0           0           0  NoEvents  
...           ...         ...         ...         ...       ...  
78372           0           0           0           0   Natural  
78373           0           0           0           0   Natural  
78374           0           0           0           0   Natural  
78375           0           0           0           0   Natural  
78376           0           0           0           0   Natural  

[78377 rows x 129 columns]

In [7]:
print(df_raw.shape)

df_raw['marker'].value_counts()

(78377, 129)


Attack      55663
Natural     18309
NoEvents     4405
Name: marker, dtype: int64

### ---> unbalanced classes

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78377 entries, 0 to 78376
Columns: 129 entries, R1-PA1:VH to marker
dtypes: float64(113), int64(15), object(1)
memory usage: 77.1+ MB


In [9]:
# Look at datatypes of coloumns.
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )
        
show_dtypes(df_raw)

R1-PA1:VH -> float64
R1-PM1:V -> float64
R1-PA2:VH -> float64
R1-PM2:V -> float64
R1-PA3:VH -> float64
R1-PM3:V -> float64
R1-PA4:IH -> float64
R1-PM4:I -> float64
R1-PA5:IH -> float64
R1-PM5:I -> float64
R1-PA6:IH -> float64
R1-PM6:I -> float64
R1-PA7:VH -> float64
R1-PM7:V -> float64
R1-PA8:VH -> float64
R1-PM8:V -> float64
R1-PA9:VH -> float64
R1-PM9:V -> float64
R1-PA10:IH -> float64
R1-PM10:I -> float64
R1-PA11:IH -> float64
R1-PM11:I -> float64
R1-PA12:IH -> float64
R1-PM12:I -> float64
R1:F -> float64
R1:DF -> float64
R1-PA:Z -> float64
R1-PA:ZH -> float64
R1:S -> int64
R2-PA1:VH -> float64
R2-PM1:V -> float64
R2-PA2:VH -> float64
R2-PM2:V -> float64
R2-PA3:VH -> float64
R2-PM3:V -> float64
R2-PA4:IH -> float64
R2-PM4:I -> float64
R2-PA5:IH -> float64
R2-PM5:I -> float64
R2-PA6:IH -> float64
R2-PM6:I -> float64
R2-PA7:VH -> float64
R2-PM7:V -> float64
R2-PA8:VH -> float64
R2-PM8:V -> float64
R2-PA9:VH -> float64
R2-PM9:V -> float64
R2-PA10:IH -> float64
R2-PM10:I -> float64
R2-P

In [10]:
# .copy() is very important otherwise its just a pointer
df = df_raw.copy()

In [11]:
def marker_to_numeric_triple(x):
        if x=='Attack': return 0
        if x=='Natural': return 1
        if x=='NoEvents': return 2
        
LABEL_MAP = {0 : "Attack", 1: "Natural", 2: "NoEvents"}

LABELNAMES = [LABEL_MAP[key] for key in LABEL_MAP]

df['marker'] = df['marker'].apply(marker_to_numeric_triple)
df

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       control_panel_log4  relay1_log  relay2_log  relay3_log  relay4_log  \
0                       0           0           0           0           0   
1                       0           0           0           0           0   
2                       0           0           0           0           0   
3                       0           0           0           0           0   
4                       0           0           0           0           0   
...                   ...         ...         ...         ...         ...   
78372                   0           0           0           0           0   
78373                   0           0           0           0           0   
78374                   0           0           0           0           0   
78375                   0           0           0           0           0   
78376                   0           0           0           0           0   

       snort_log1  snort_log2  snort_log3  snort_log4  marker  
0               0           0           0           0       2  
1               0           0           0           0       2  
2               0           0           0           0       2  
3               0           0           0           0       2  
4               0           0           0           0       2  
...           ...         ...         ...         ...     ...  
78372           0           0           0           0       1  
78373           0           0           0           0       1  
78374           0           0           0           0       1  
78375           0           0           0           0       1  
78376           0           0           0           0       1  

[78377 rows x 129 columns]

In [12]:
# Check for infinite float entries
# "np.finfo(np.float64).max" returns the maximum finite value that can be represented by a 64-bit floating point number
# "df.values >= np.finfo(np.float64).max" returns a bollean array where "np.finfo(np.float64).max" is checked
# "np.where(df.values >= np.finfo(np.float64).max)" returns a array of indices where "df.values >= np.finfo(np.float64).max" is true

np.where(df.values >= np.finfo(np.float64).max)

(array([  738,   739,   753, ..., 78352, 78353, 78353], dtype=int64),
 array([26, 26, 26, ..., 55, 26, 55], dtype=int64))

### ----> we have infinite float entries

In [13]:
# Check for NaN values
np.isnan(df.values.any())

False

### ----> we have no NaN values in the dataframe

In [14]:
# Replace infinite updated data with nan

# "[np.inf, -np.inf]" specifies the values to be replaced
# "np.nan" specifies the value to replace the infinite values with.
# "inplace=True" modifies the Dataframe in place, so that the original df is updated the new values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

(72073, 129)


0    51445
1    16223
2     4405
Name: marker, dtype: int64

### ----> Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful due to the fact that there are still 72000 rows

In [15]:
y = df['marker']

X = df.drop(['marker'],axis =1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((43243, 128), (43243,), (14415, 128), (14415,), (14415, 128), (14415,))

In [17]:
train_data = xgb.DMatrix(X_train, label=y_train)
val_data = xgb.DMatrix(X_val, label=y_val)

In [19]:
params = [
        ("num_class", 10), ("eval_metric", "merror")
    ]

n_rounds = 600
early_stopping = 50

# build the model
results = {}
model = xgb.train(params, 
                    train_data, 
                    num_boost_round=100, 
                    evals=[(val_data, 'val')], 
                    evals_result=results)

[0]	val-merror:0.25834
[1]	val-merror:0.25467
[2]	val-merror:0.24918
[3]	val-merror:0.24766
[4]	val-merror:0.24509
[5]	val-merror:0.24440
[6]	val-merror:0.24398
[7]	val-merror:0.24308
[8]	val-merror:0.24107
[9]	val-merror:0.24107
[10]	val-merror:0.23940
[11]	val-merror:0.23829
[12]	val-merror:0.23711
[13]	val-merror:0.23482
[14]	val-merror:0.23392
[15]	val-merror:0.23323
[16]	val-merror:0.23073
[17]	val-merror:0.22907
[18]	val-merror:0.22712
[19]	val-merror:0.22525
[20]	val-merror:0.22331
[21]	val-merror:0.22234
[22]	val-merror:0.22171
[23]	val-merror:0.21991
[24]	val-merror:0.21880
[25]	val-merror:0.21845
[26]	val-merror:0.21818
[27]	val-merror:0.21734
[28]	val-merror:0.21658
[29]	val-merror:0.21478
[30]	val-merror:0.21429
[31]	val-merror:0.21325
[32]	val-merror:0.21159
[33]	val-merror:0.21013
[34]	val-merror:0.20909
[35]	val-merror:0.20888
[36]	val-merror:0.20839
[37]	val-merror:0.20826
[38]	val-merror:0.20728
[39]	val-merror:0.20735
[40]	val-merror:0.20610
[41]	val-merror:0.20520
[4

In [22]:
test_data = xgb.DMatrix(X_test)

predictions = model.predict(test_data)

accuracy = metrics.accuracy_score(y_test, predictions)

In [23]:
print(accuracy)

0.8372528616024973


In [ ]:
def evaluate_model(model, X_test, y_test, labelmap):
    """
    Evaluate a multiclass classification model using various metrics.
    
    Args:
        model (sklearn estimator): The model to evaluate.
        X_test (pandas dataframe): The test features.
        y_test (pandas series): The true labels
        labelmap (dict): The encoding of the names of the labels that are passed to report and matrix
        
    Returns:
        sklearn confusionmatrix
    """
    
    labelnames = [labelmap[key] for key in labelmap]
    pos_label = "Attack"
    
    # Predict the test labels using the trained model
    y_pred = model.predict(X_test)
    
    # map encoded labels to original labels
    y_true = [labelmap[i] for i in y_test]
    y_pred = [labelmap[i] for i in y_pred]
    
    # Compute the confusion matrix and print it
    confusionmatrix = confusion_matrix(y_true, y_pred, labels=labelnames)
    print("Confusion matrix:")
    print(confusionmatrix)
    
    # Compute the classification report and print it
    report = classification_report(y_true, y_pred, target_names=labelnames)
    print("Classification report:")
    print(report)
    
    return confusionmatrix, report

In [ ]:
cm, rp = evaluate_model(bst, X_test, y_test, LABEL_MAP)

In [ ]:
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELNAMES, yticklabels=LABELNAMES)

# UPSAMPLING

In [ ]:
df_minority_NoEvents = df[df['marker'] == 0]
df_minority_Natural = df[df['marker'] == 1]
df_not_minority = df[df['marker'] == 2] 

In [ ]:
#upsample the minority classes
df_minority_NoEvents_upsampled = resample(df_minority_NoEvents,random_state=42,n_samples=51445,replace=True)
df_minority_Natural_upsampled = resample(df_minority_Natural,random_state=42,n_samples=51445,replace=True)

In [ ]:
print(df_minority_NoEvents_upsampled.shape)
print(df_minority_Natural_upsampled.shape)

In [ ]:
#concatenate the upsampled dataframes
df_upsampled = pd.concat([df_minority_NoEvents_upsampled, df_minority_Natural_upsampled, df_not_minority ])
#df_upsampled

In [ ]:
y = df_upsampled['marker']

X = df_upsampled.drop(['marker'],axis =1)

In [ ]:
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

print("upsampled Train data:")
print(X_train.shape, y_train.shape)

print("untouched Test data:")
print(X_test.shape, y_test.shape)

In [ ]:
#RF with Upsampled Data
model_rf_upsampled = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
%time model_rf_upsampled.fit(X_train, y_train) 

In [ ]:
cm, rp = evaluate_model(model_rf_upsampled, X_test, y_test, LABEL_MAP)

In [ ]:
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELNAMES, yticklabels=LABELNAMES)